- Zbiór danych iris podziel na 5 losowych podzbiorów.
- Dla każdego z nich wygeneruj reguły decyzyjne.
- Posortuj otrzymane reguły względem pokrycia.
- Przeanalizuj otrzymane wyniki.


In [3]:
from sklearn.datasets import load_iris
import pandas as pd
from helpers import sequential_covering

iris = load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['Decyzja'] = iris.target

partition_count = 5
max_rules = 5

rules_per_partition = []

for i in range(partition_count):
    df_partition = df.sample(frac=0.7, random_state=i) # losowo wybieramy 70% wierszy.

    #dyskretyzacja
    X_dysk = df_partition.copy()

    for column in iris.feature_names:
        X_dysk[column] = pd.qcut(df[column], q=3, duplicates='drop').astype(str) # podział danych na 3 kwantyle.

    X = X_dysk[iris.feature_names]
    y = df_partition["Decyzja"]

    rules = sequential_covering(X, y, max_rules=max_rules)
    rules_per_partition.append(rules)

    print(f"przedział {i+1}:")
    for regula, class_rules, liczba_pokrytych in sorted(rules, key=lambda x: x[2], reverse = True):
        warunki = " AND ".join([f"{atr} = {wart}" for atr, wart in regula])
        print(f" {warunki} -> {iris.target_names[class_rules]} (pokryte: {liczba_pokrytych})")
    print("")

przedział 1:
 petal width (cm) = (1.6, 2.5] -> virginica (pokryte: 34)
 petal length (cm) = (0.999, 2.633] -> setosa (pokryte: 33)
 petal width (cm) = (0.099, 0.867] -> setosa (pokryte: 33)
 petal width (cm) = (0.867, 1.6] AND petal length (cm) = (2.633, 4.9] -> versicolor (pokryte: 33)
 petal length (cm) = (2.633, 4.9] AND sepal length (cm) = (6.3, 7.9] -> versicolor (pokryte: 6)

przedział 2:
 petal length (cm) = (0.999, 2.633] -> setosa (pokryte: 33)
 petal width (cm) = (0.099, 0.867] -> setosa (pokryte: 33)
 petal length (cm) = (2.633, 4.9] AND sepal length (cm) = (6.3, 7.9] -> versicolor (pokryte: 7)
 sepal width (cm) = (3.2, 4.4] AND sepal length (cm) = (6.3, 7.9] -> virginica (pokryte: 5)
 sepal length (cm) = (4.2989999999999995, 5.4] AND petal length (cm) = (2.633, 4.9] -> versicolor (pokryte: 4)

przedział 3:
 petal length (cm) = (0.999, 2.633] -> setosa (pokryte: 37)
 petal width (cm) = (0.099, 0.867] -> setosa (pokryte: 37)
 sepal width (cm) = (1.999, 2.9] AND petal length (